In [ ]:
import numpy as np
import tables as tb
import matplotlib.pyplot as plt
plt.rcParams.update({"text.usetex": True, "font.size":20, "font.family": "sans"})

In [ ]:
# proton = tb.open_file('proton_Espec.h5')
# iron = tb.open_file('iron_Espec.h5')
# proton_aab = tb.open_file('proton_Aab.h5')
# iron_aab = tb.open_file('iron_Aab.h5')

proton = tb.open_file('new_proton.h5')
iron = tb.open_file('new_iron.h5')

In [ ]:
len(proton.root.bins.r_bins)

In [ ]:
def get_bin(x, bins):
    ix = np.nanargmin(np.abs(bins - x))
    if ix >= len(bins) -1:
        ix = len(bins)-2
    return ix


def get_spec(t_h5, ptype, r, psi, 
             lgE=(18.5, 20), cosTheta=(0.5, 1), Xmax=(0, 2000)):
    
    bins = t_h5.root.bins
    table = t_h5.root.event
    
    lgE_min, lgE_max = lgE
    cT_min, cT_max = cosTheta
    Xmax_min, Xmax_max = Xmax
    
    ir = get_bin(r, bins.r_bins.read())
    ipsi = get_bin(psi, bins.psi_bins.read())
    if ipsi == 0:
        ipsi = 1
    if ipsi >= len(bins.psi_bins.read()) - 2:
        ipsi = -2
    if ptype == 'muon':
        specs = t_h5.root.muon_spec
    elif ptype == 'electron':
        specs = t_h5.root.electron_spec
    elif ptype == 'photon':
        specs = t_h5.root.photon_spec

    cut = f'(Xmax >= {Xmax_min}) & (Xmax < {Xmax_max}) &\
            (cosTheta >= {cT_min}) & (cosTheta < {cT_max}) &\
            (lgE >= {lgE_min}) & (lgE < {lgE_max})'


    nE = len(bins.lgE_pkin_bins)-1
    spec = np.array([specs[t['it']][ir, ipsi, :] for t in table.where(cut)])
    
    if spec.shape[0] == 0:
        print("No showers!")
        return np.zeros((1, nE))
        
    lgE, dlgE, mE, dE = get_E(t_h5)

    return spec/dlgE

def get_E(t_h5):     
    bins = t_h5.root.bins

    lgEk = bins.lgE_pkin_bins.read()
    Ek = 10**lgEk
    mlgE = (lgEk[1:] + lgEk[:-1])/2
    dlgE = lgEk[1:] - lgEk[:-1]
    mE = (Ek[1:] + Ek[:-1])/2
    dE = Ek[1:] - Ek[:-1]
    
    return mlgE, dlgE, mE, dE

def plot_spec(t_h5, ptype, ax, cuts, norm=True, **pltkws):
    
    lgE, _, _, _ = get_E(proton)

    spec = get_spec(t_h5, ptype, **cuts)
    if norm:
        spec = spec/spec.sum(axis=1)[:, None]

    pl = ax.plot(lgE, spec.mean(axis=0), **pltkws)
#     ax.plot(lgE, spec.T, color=pl[0].get_color(), ls=pl[0].get_ls(), alpha=1/(len(spec)+1))
#     ax.errorbar(lgE, spec.mean(axis=0), yerr=spec.std(axis=0)/np.sqrt(len(spec)), **pltkws)


In [ ]:
EVEM = 380e6
ECH = 58e6

def Lwcd(chi):
    return 1/(np.abs(np.cos(chi)) + 0.42*np.sin(chi))

def calculate_ave_E(spec, E, low=0, high=1e12):
    mask = (E>low) & (E<high)
    return (spec*E)[mask].sum()/spec.sum()

def calculate_alpha(spec_el, spec_ph, E, low=0, high=1e12, theta=0):
    Ee = calculate_ave_E(spec_el, E, low, high)
    Eph = calculate_ave_E(spec_ph, E, low, high)
    Ne = spec_el.sum()
    Nph = spec_ph.sum()
    rph = Nph/Ne
    wcd_e_eff = 0.75
    wcd_ph_eff = 0.823
    ssd_e_eff = 0.97
    ssd_ph_eff = 0.034
    alpha = (Ee*Ne/EVEM*wcd_e_eff + Eph*Nph/EVEM*wcd_ph_eff) / (Ne*ssd_e_eff + Nph*ssd_ph_eff)
    return alpha/Lwcd(theta)

def calculate_beta(spec_mu, E, theta=0):
    Sf1 = calculate_ave_E(spec_mu, E, low=ECH, high=EVEM)/EVEM
    muon_wcd_eff1 = 0.199#??
    muon_wcd_eff2 = 1
    f2 = spec_mu[E>EVEM].sum()/spec_mu.sum() 
    return muon_wcd_eff1 * Sf1/Lwcd(theta) + muon_wcd_eff2*f2
    
    

In [ ]:
h5 = proton
lgE, dlgE, E, dE = get_E(h5)
cuts = {'r': 1000, 'psi': 0,  'lgE': (18.9, 19.1), 'cosTheta': (0.9, 1),'Xmax': (600, 900)}

spec_el = get_spec(h5, 'electron', **cuts).mean(axis=0)
spec_ph = get_spec(h5, 'photon', **cuts).mean(axis=0)
spec_mu = get_spec(h5, 'muon', **cuts).mean(axis=0)

Ee = calculate_ave_E(spec_el, E)
Eph = calculate_ave_E(spec_ph, E)

Ne = spec_el.sum()
Nph = spec_ph.sum()

rph = Nph/Ne

print("Average electron energy %.0f MeV" % (Ee/1e6))
print("Average photon energy %.0f MeV" % (Eph/1e6))
print("Nph/Nel %.2f" % (rph))

print("alpha = %.2f" % calculate_alpha(spec_el, spec_ph, E, theta=0))

print("beta = %.2f" % calculate_beta(spec_mu, E))

<font size=4>
$S^{em}_w/S^{em}_s = (A_w N_e E_e/E_{vem} + N_\gamma E_\gamma/E_{vem})/(A_s N_e) = A_w/A_s (E_e/E_{vem} + N_\gamma/N_e E_\gamma/E_{vem})$   
$\alpha = 0.38 S^{em}_w/S^{em}_s =(E_e/E_{vem} + N_\gamma/N_e E_\gamma/E_{vem})$

In [ ]:
r = 750
cosTheta = 0.95

f, ax = plt.subplots(1, figsize=(10, 7))

h5 = proton
lgE, dlgE, E, dE = get_E(h5)

for lgEcr, ls in zip([18.5, 19, 19.5], ['-', '--', ':']):

    cuts = {'r': r, 'psi': 0,
            'lgE': (lgEcr-0.1, lgEcr+0.1), 
            'cosTheta': (cosTheta-0.05, cosTheta+0.05),
            'Xmax': (600, 900)}

    spec_el = get_spec(h5, 'electron', **cuts)
    spec_el = spec_el.mean(axis=0)
    spec_ph = get_spec(h5, 'photon', **cuts)
    spec_ph = spec_ph.mean(axis=0)

    Ee = calculate_ave_E(spec_el, E)
    Eph = calculate_ave_E(spec_ph, E)

    Ne = spec_el.sum()
    Nph = spec_ph.sum()

    rph = Nph/Ne
    alpha = calculate_alpha(spec_el, spec_ph, E)

    print('================================')
    print('lgE = %.2f' %lgEcr)
    print("~vertical showers")
    print("at r = %.0f m" % r)
    print("rph = %.2f" % (rph))
    print("Average electron energy %.0f MeV" % (Ee/1e6))
    print("Average photon energy %.0f MeV" % (Eph/1e6))
    print("Evem = %.0f MeV" % (EVEM/1e6))
    print("alpha = %.2f VEM/MIP" % calculate_alpha(spec_el, spec_ph, E))

    ax.plot(lgE, spec_el/Ne, 'r-', label='electron lgE = %.1f $\\alpha = %.2f$' %(lgEcr, alpha), ls=ls)
    ax.plot(lgE, spec_ph/Nph, 'y-', label='photon lgE = %.1f' %lgEcr, ls=ls)
    
ax.set_xlabel('E [eV]')
ax.set_ylabel('dN/dlgE')
ax.set_yscale('log')  
ax.legend(fontsize=15)

ax.set_title('r = %.0f m' % r)

In [ ]:
cts = np.linspace(0.5, 1, 6)

f, ax = plt.subplots(1, figsize=(7, 5))

for h5, prim in zip([proton, iron], ['p','Fe']):
    betas = []
    for ct in cts:

        cuts = {'r': 1000, 'psi': 0,
                'lgE': (19.4, 19.6), 
                'cosTheta': (ct-0.05, ct+0.05),
                'Xmax': (600, 1100)}

        spec = get_spec(h5, 'muon', **cuts)
        spec = spec.mean(axis=0)
        
        lgE, dlgE, E, dE = get_E(h5)

        beta = calculate_beta(spec, E, theta=np.arccos(ct))
        betas.append(beta)
    
    if prim == 'p':
        color='b'
    else:
        color = 'r'
    ax.plot(1/cts, betas, color=color, ls='-', marker='o', label=prim)
    
ax.legend()
ax.set_xlabel('$\\sec{\\theta}$')
ax.set_ylabel('$\\beta$')
ax.set_title('lgE = 19.5, r = 1000 m', fontsize=18)
# plt.savefig('beta_vs_sect_p_Fe.png', bbox_inches='tight', dpi=300)

In [ ]:
Evem = 380e6
Ech = 58e6

rs = proton.root.bins.r_bins.read()[:-1]

f, ax = plt.subplots(1, figsize=(7, 5))

for h5, prim in zip([proton, iron], ['p','Fe']):
    betas = []
    for r in rs:
        cuts = {'r': r, 'psi': 0,
                'lgE': (19.4, 19.6), 
                'cosTheta': (0.95, 1),
                'Xmax': (600, 1200)}

        spec = get_spec(h5, 'muon', **cuts)
        spec = spec.mean(axis=0)
        
        lgE, dlgE, E, dE = get_E(h5)

        beta = calculate_beta(spec, E, theta=np.arccos(ct))
        betas.append(beta)
    
    
    if prim == 'p':
        color='b'
    else:
        color = 'r'
    ax.plot(rs, betas, color=color, ls='-', marker='o', label=prim)
    
ax.legend()
ax.set_xlabel('$r$ [m]')
ax.set_ylabel('$\\beta$')
ax.set_title('lgE = 19.5, theta = 0 deg', fontsize=18)
# plt.savefig('C_p_Fe_vs_r.png', bbox_inches='tight', dpi=300)

In [ ]:
r = 1000
lgEcr = 19
cts = np.linspace(0.5, 1, 7)

f, ax = plt.subplots(1, figsize=(7, 5))

for h5, prim in zip([proton, iron], ['p','Fe']):
    alphas = []
    for ct in cts:
        cuts = {'r': r, 'psi': 0,
                'lgE': (lgEcr-0.1, lgEcr+0.1), 
                'cosTheta': (ct-0.05, ct+0.05),
                'Xmax': (0, 1400)}

        spec_el = get_spec(h5, 'electron', **cuts)
        spec_el = spec_el.mean(axis=0)
        spec_ph = get_spec(h5, 'photon', **cuts)
        spec_ph = spec_ph.mean(axis=0)
        
        lgE, dlgE, E, dE = get_E(h5)

        alpha = calculate_alpha(spec_el, spec_ph, E)#, theta=np.arccos(ct)+0)
        alphas.append(alpha)
        
    if prim == 'p':
        color='b'
    else:
        color = 'r'
    ax.plot(1/cts, alphas, color=color, ls='-', marker='s', label=prim)
    
ax.legend()
ax.set_xlabel('$\\sec{\\theta}$')
ax.set_ylabel('$\\alpha$')
ax.set_title('lgE = %.1f, r = %.0f m' % (lgEcr, r), fontsize=16)


In [ ]:

lgEcr = 19
rs = proton.root.bins.r_bins.read()[:-1]
cosTheta = 0.95

f, ax = plt.subplots(1, figsize=(7, 5))

for h5, prim in zip([proton, iron], ['p','Fe']):
    alphas = []
    for r in rs:

        cuts = {'r': r, 'psi': 0,
                'lgE': (lgEcr-0.1, lgEcr+0.1), 
                'cosTheta': (cosTheta-0.05, cosTheta+0.05),
                'Xmax': (600, 1100)}

        spec_el = get_spec(h5, 'electron', **cuts)
        spec_el = spec_el.mean(axis=0)
        spec_ph = get_spec(h5, 'photon', **cuts)
        spec_ph = spec_ph.mean(axis=0)
        
        lgE, dlgE, E, dE = get_E(h5)

        alpha = calculate_alpha(spec_el, spec_ph, E, theta=np.arccos(cosTheta))
        alphas.append(alpha)
        
        
    if prim == 'p':
        color='b'
    else:
        color = 'r'
    ax.plot(rs, alphas, color=color, ls='-', marker='s', label=prim)
    
ax.legend()
ax.set_xlabel('r [m]')
ax.set_ylabel('$\\alpha = 0.38 S^{em}_W/S^{em}_S$')
ax.set_title('lgE = %.1f, theta = %.0f deg' % (lgEcr, np.rad2deg(np.arccos(cosTheta))), fontsize=16)


In [ ]:
r = 1000
cosTheta = 0.9
lgEs = np.linspace(18.5, 20, 7)

f, ax = plt.subplots(1, figsize=(7, 5))

for h5, prim in zip([proton, iron], ['p','Fe']):
    alphas = []
    for lgEcr in lgEs:

        cuts = {'r': r, 'psi': 0,
                'lgE': (lgEcr-0.05, lgEcr+0.05), 
                'cosTheta': (ct-0.05, ct+0.05),
                'Xmax': (600, 1100)}

        spec_el = get_spec(h5, 'electron', **cuts)
        spec_el = spec_el.mean(axis=0)
        spec_ph = get_spec(h5, 'photon', **cuts)
        spec_ph = spec_ph.mean(axis=0)
        
        lgE, dlgE, E, dE = get_E(h5)

        alpha = calculate_alpha(spec_el, spec_ph, E, theta = np.arccos(ct))
        alphas.append(alpha)
        
    if prim == 'p':
        color='b'
    else:
        color = 'r'
    ax.plot(lgEs, alphas, color=color, ls='-', marker='s', label=prim)
    
ax.legend()
ax.set_xlabel('lgE')
ax.set_ylabel('$\\alpha$')
ax.set_title('theta = %.0f deg, r = %.0f m' % (np.rad2deg(np.arccos(cosTheta)), r), fontsize=16)
